In [ ]:
import numpy
import pandas
import requests
from bs4 import BeautifulSoup
import re
from requests_html import HTMLSession
from urllib.parse import parse_qs

# 1. Get available cities
# 2. Get available businesses
# 3. Get available restaurants in each city
# 4. Get urls for each business
# 5. Get website links for each business
# 6. Get reviews and number of reviews for each
# 7. Get phone numbers
# 8. Get email_ids
# 9. Get addresses

##  SAVE ALL THESE IN CSV SHEETS

##### 

#### 

###  Generating city names and generating urls with them

In [ ]:
def city_names_extracter(country):
    city_names_data = requests.get(("https://www.yelp.co.uk/locations/countries/{}").format(country))
    city_names_data.status_code

    data_city_names = BeautifulSoup(city_names_data.content, 'html.parser')

    cities = []
    for i in data_city_names.select('li a'):
        if i.text!= "About Yelp":
            cities.append(i.text)
        else:
            break

    return cities

In [ ]:
cities = city_names_extracter('nz')
cities_copy = cities
len(cities)

In [ ]:
def city_url_generator(cities_list):
    city_wise_urls =[]
    edited_cities_list = []
    for i in cities_list:
        if i not in edited_cities_list:
                edited_cities_list.append(i)
                
        if " " in i:
            cities_list.remove(i)
            s = '%2'
            edit = i.strip().split()
            joined = s.join(edit)
            cities_list.append(joined)
            
    for j in edited_cities_list:
        url = "https://www.yelp.com/search?cflt=restaurants&find_loc={}%2C+New+Zealand".format(j)
        city_wise_urls.append(url)
    return city_wise_urls

In [ ]:
all_citywise_urls = city_url_generator(cities_copy)
all_city_wise_urls_copy = all_citywise_urls
len(city_url_generator(cities_copy))
all_city_wise_urls_copy

## Extracting number of pages available for each city

In [ ]:
def pages_for_each_city(cities_urls):
    total_pages = []
    for t in cities_urls:
        h = requests.get(t)
        sou = BeautifulSoup(h.content, 'html.parser')
        pages_count = sou.find_all('span')[-21].text[-1]
        if bool(re.search(r'\d', pages_count)):
            total_pages.append(pages_count)
    return total_pages

In [ ]:
# bool(re.search(r'\d', inputString))
pages_for_each_city(all_city_wise_urls_copy)
total_pages_citywise = pages_for_each_city(all_city_wise_urls_copy)
total_pages_citywise_copy = total_pages_citywise

In [ ]:
total_pages_citywise_copy

In [ ]:
def businesses_name_extractor(cities_names_list):
    all_names = []
    basic_urls_citywise =[]
    for n in cities_names_list:
        basic_urls_each_city =[]
        names_here = []
        url = "https://www.yelp.com/search?cflt=restaurants&find_loc={}%2C+New+Zealand".format(n)
        req_url = requests.get(url)
        soup_url = BeautifulSoup(req_url.content, 'html.parser')
        here = soup_url.select('h4 span a')
        for all_in_here in here:
            names_here.append(all_in_here.text)
        available_pages = soup_url.find_all('span')[-21].text[-1]
        start =10
        try:
            while start< (10*int(available_pages)):
                further_urls ="https://www.yelp.com/search?cflt=restaurants&find_loc={}&start={}".format(n, start)
                basic_urls_each_city.append(further_urls)
                req_further = requests.get(further_urls)
                soup_further = BeautifulSoup(req_further.content, 'html.parser')
                further_names = soup_further.select('h4 span a')
                for each_further in further_names:
                    names_here.append(each_further.text)
                start = start+10
        except:
            pass
        basic_urls_citywise.append(basic_urls_each_city)
        all_names.append(names_here)
    return all_names, basic_urls_citywise

In [ ]:
restaurants_names = businesses_name_extractor(cities_copy)
restaurants_names_copy = restaurants_names

In [ ]:
basic_urls_citywise = restaurants_names_copy[1]

In [ ]:
basic_urls_citywise_copy = basic_urls_citywise
basic_urls_citywise_copy

In [ ]:
def filtering_cities_with_businesses(urls_citywise):
    remaining_cities = []
    not_available = []
    for out in urls_citywise:
        try:
            rem = out[0].split('find_loc=')[1].split('&')[0]
            remaining_cities.append(rem)
        except:
            not_available.append(out)
            continue
    return remaining_cities, not_available

In [ ]:
filtering_cities_with_businesses(basic_urls_citywise_copy)
cities_with_restaurants = filtering_cities_with_businesses(basic_urls_citywise_copy)[0]
cities_without_restaurants  = filtering_cities_with_businesses(basic_urls_citywise_copy)[1]

In [ ]:
len(cities_with_restaurants)

In [ ]:
cities_without_restaurants

### extracting biz elements from href elements and generate urls

In [ ]:
def biz_extractor(given_basic_urls):
    extracted = []
    for basic in given_basic_urls:
        basic_extracted = []
        for each_basic in basic:
            req = requests.get(each_basic)
            print(req.status_code)
            status = req.status_code
            if status==200:
                print('it is 200')
            if status==503:
                print('503, breaked')
                break
            try:
                soup = BeautifulSoup(req.content, 'html.parser')
                selected_a = soup.select('a')
                for each_a in selected_a:
                    each_hrefs = each_a.attrs.get('href')
                    print(each_hrefs)
                    string = '/biz/'
                    try:
                        if string in str(each_hrefs):
                            print(each_hrefs)
                            try:
                                stripped_element = str(each_hrefs).split('?')[0]
                                print(stripped_element, 'tried')
                            except:
                                stripped_element = each_hrefs
                                print(stripped_element, 'excepted')
                            urls_unique = "https://www.yelp.com"+str(stripped_element)
                            print(urls_unique)
                            if not str(urls_unique) in basic_extracted:
                                if '#popup:platform' not in str(urls_unique):
                                    print('popup found')
                                    basic_extracted.append(urls_unique)
                                    print('added as unique')
                                        

                    except:
                        continue
            except:
                print('503')
 
        print('looped')
        extracted.append(basic_extracted)
    return extracted

In [ ]:
extracted_biz_links = biz_extractor(basic_urls_citywise_copy)

In [ ]:
extracted_biz_links_copy = extracted_biz_links

In [ ]:
extracted_biz_links_copy
# len(extracted_biz_links_copy)

In [ ]:
def redir_extraction(unique_urls_all):
    actual_all_hrefs = []
    actual_excepted = []
    for outer in range(len(unique_urls_all)):
        inner_hrefs = []
        inner_excepted = []
        print(outer)
        for inner in range(len(unique_urls_all[outer])):
            actual_url = unique_urls_all[outer][inner]
            actual_req = requests.get(actual_url)
            print(actual_req.status_code)
            actual_soup = BeautifulSoup(actual_req.content, 'html.parser')
            a_actual = actual_soup.select('a')
            for act in a_actual:
                href_actual = act.attrs.get('href')
                try:
                    if "/biz_redir?url=" in href_actual:
                        print(href_actual)
                        inner_hrefs.append(href_actual)
                    else:
                        inner_excepted.append(href_actual)

                except:
                    continue
               
        actual_excepted.append(inner_excepted)
        print(inner_hrefs)
        actual_all_hrefs.append(inner_hrefs)
    return actual_all_hrefs

In [ ]:
redirect_links = redir_extraction(extracted_biz_links_copy)

In [ ]:
redirect_links_copy = redirect_links

In [ ]:
redirect_links

In [ ]:
len(redirect_links_copy)

In [ ]:
redirect_links_copy

In [ ]:
a = [i for i in redirect_links_copy if len(i)!=0] 

In [ ]:
len(a)